# 1. 환경 셋업

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import boto3
# import json

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

python path: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery is added
sys.path:  ['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/joohyery/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery']


In [28]:
# 필요한 변수들 정의

region = "us-west-2"
nova_pro_model_id = "us.amazon.nova-pro-v1:0"
claude3_7_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

client = boto3.client(service_name="bedrock-runtime", region_name=region)


# 2. 샘플 포트폴리오 경로 로딩 하기

In [4]:
current_folder = os.getcwd()
print(current_folder)

/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate


In [5]:
sample_path = os.path.join(os.getcwd(), 'samples')

# 해당 폴더 내 파일/디렉토리 리스트 출력
if os.path.exists(sample_path):
    for filename in os.listdir(sample_path):
        print(filename)
else:
    print("'samples' 폴더가 현재 위치에 없습니다.")

target = "None"

sample1.jpg


# 3. 리플렛 OCR

In [20]:
# 한국어 리플렛에서 번역을 위한 모든 텍스트를 체계적으로 추출할 수 있도록 시스템 프롬프트 작성
ocr_system_prompt = """You are a professional text extraction specialist. Your task is to carefully analyze Korean leaflets/brochures and extract ALL text content for English translation purposes.

## Instructions:
1. Examine the provided Korean leaflet thoroughly
2. Extract ALL visible text, including:
   - Headlines and titles
   - Body text and paragraphs
   - Captions and descriptions
   - Contact information (phone numbers, addresses, emails, websites)
   - Small print and disclaimers
   - Menu items, prices, or product listings
   - Date and time information
   - Terms and conditions

3. DO NOT extract text that appears within images, logos, graphics, or decorative elements
   - Focus only on regular text content that is part of the leaflet's main text layout
   - Skip any text that is embedded in photographs, illustrations, or logo designs

4. Organize the extracted text in a logical order:
   - Start with main headings/titles
   - Follow the visual flow of the leaflet (left to right, top to bottom)
   - Group related content together
   - Clearly separate different sections

5. Present the text in the original Korean language - do NOT translate anything
6. If text is unclear or partially visible, note it as [UNCLEAR: approximate text]
7. Maintain the original formatting structure as much as possible

Your goal is to ensure no regular text content is missed while excluding text within images/graphics, so that the subsequent English translation will be complete and accurate for the main textual content only.
"""


In [7]:
import base64
from PIL import Image
import io
from botocore.exceptions import ClientError

# Bedrock Converse API에서 content 필드에 넣을 수 있는 이미지 크기 제한은 3.75MB라 이미지를 압축하는 코드가 필요
def compress_image(image_path, max_size_mb=5):
    """
    이미지를 지정된 크기 이하로 압축하는 함수
    
    Args:
        image_path (str): 원본 이미지 경로
        max_size_mb (int): 최대 크기 (MB)
    
    Returns:
        bytes: 압축된 이미지 바이트 데이터
        str: 이미지 포맷
    """
    max_size_bytes = max_size_mb * 1024 * 1024  # MB를 바이트로 변환
    
    # 이미지 열기
    with Image.open(image_path) as img:
        # 원본 포맷 확인
        original_format = img.format.lower() if img.format else 'png'
        
        # RGB로 변환 (JPEG 저장을 위해)
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
            
        # 초기 품질 설정
        quality = 95
        
        while quality > 10:  # 최소 품질 10까지 시도
            # 이미지를 메모리 버퍼에 저장
            buffer = io.BytesIO()
            
            # JPEG로 압축 (가장 효율적)
            img.save(buffer, format='JPEG', quality=quality, optimize=True)
            buffer.seek(0)
            
            # 압축된 크기 확인
            compressed_size = len(buffer.getvalue())
            
            if compressed_size <= max_size_bytes:
                print(f"이미지 압축 완료: {compressed_size:,} bytes (품질: {quality})")
                return buffer.getvalue(), 'jpeg'
            
            # 품질을 10씩 감소
            quality -= 10
        
        # 품질을 낮춰도 안 되면 크기를 줄임
        print("품질 조정으로 압축 실패. 이미지 크기를 조정합니다...")
        
        # 이미지 크기 단계적 축소
        scale_factors = [0.8, 0.6, 0.4, 0.2]
        
        for scale in scale_factors:
            # 이미지 크기 축소
            new_width = int(img.width * scale)
            new_height = int(img.height * scale)
            resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            # 압축된 이미지 저장
            buffer = io.BytesIO()
            resized_img.save(buffer, format='JPEG', quality=85, optimize=True)
            buffer.seek(0)
            
            compressed_size = len(buffer.getvalue())
            
            if compressed_size <= max_size_bytes:
                print(f"이미지 리사이징 및 압축 완료: {compressed_size:,} bytes (크기: {new_width}x{new_height})")
                return buffer.getvalue(), 'jpeg'
        
        # 최후의 수단: 매우 작은 크기로 축소
        resized_img = img.resize((800, 600), Image.Resampling.LANCZOS)
        buffer = io.BytesIO()
        resized_img.save(buffer, format='JPEG', quality=70, optimize=True)
        buffer.seek(0)
        
        print(f"최종 압축 완료: {len(buffer.getvalue()):,} bytes (800x600, 품질: 70)")
        return buffer.getvalue(), 'jpeg'



In [34]:
def ocr(target_path):
    try:
        # 이미지 압축
        print(f"원본 이미지 크기 확인 중...")
        original_size = os.path.getsize(target_path)
        print(f"원본 크기: {original_size:,} bytes ({original_size/1024/1024:.1f} MB)")
        
        if original_size > 3.75 * 1024 * 1024:  # 5MB 초과
            print("이미지가 3.75MB를 초과합니다. 압축을 시작합니다...")
            image_bytes, image_format = compress_image(target_path)
        else:
            # 3.75MB 이하면 원본 사용
            with open(target_path, "rb") as image_file:
                image_bytes = image_file.read()
            
            # 원본 포맷 확인
            file_extension = os.path.splitext(target_path)[1].lower()
            if file_extension == '.png':
                image_format = 'png'
            elif file_extension in ['.jpg', '.jpeg']:
                image_format = 'jpeg'
            else:
                image_format = 'png'

        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "text": f"<target_image>\n{target_path}\n</target_image>\n\n제공된 리플렛의 텍스트를 추출하세요 (원본 크기가 커서 압축했을 수 있습니다)"
                    },
                    {
                        "image": {
                            "format": image_format,
                            "source": {
                                "bytes": image_bytes
                            }
                        }
                    }
                ]
            }
        ]

        print("Bedrock API 호출 중...")
        response = client.converse(
            modelId=nova_pro_model_id,
            messages=messages,
            system=[
                {
                    "text": ocr_system_prompt
                },
                {
                    "cachePoint": {
                        "type": "default"
                    }
                }
            ],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.1,
                "topP": 0.9
            }
        )

        return response
    
    except FileNotFoundError:
        print(f"이미지 파일을 찾을 수 없습니다: {target_path}")
        return None
    except ClientError as e:
        print(f"Bedrock API 오류: {e}")
        return None
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")
        return None

## Bedrock API 이미지 제한을 해결하기 위한 파일 전처리

1. 파일 유형 판단
- is_image(): PIL을 사용해 이미지 파일인지 검증
- is_document(): 파일 확장자로 문서 파일인지 확인

2. 이미지 처리 단계
- 크기 확인: 원본 이미지의 너비, 높이, 파일 크기 확인
- 너비 조정: 8,000px 초과 시 비례적으로 리사이즈
- 높이 분할: 8,000px 초과 시 여러 개 이미지로 수직 크롭
- 압축: 각 이미지를 3.75MB 이하로 JPEG 압축 (초기 품질 95%에서 시작 -> 파일 크기가 3.75MB를 초과하면 품질을 5%씩 감소 -> 최소 품질 10%까지 압축)


In [14]:
import os
from typing import List, Tuple
from PIL import Image
import io
from botocore.exceptions import ClientError

def file_size_mb(path: str) -> float:
    """파일 크기를 MB 단위로 반환"""
    return os.path.getsize(path) / (1024 * 1024)

def is_image(path: str) -> bool:
    """파일이 이미지인지 확인"""
    try:
        with Image.open(path) as img:
            img.verify()
        return True
    except Exception:
        return False

def is_document(path: str) -> bool:
    """파일이 문서인지 확인"""
    ext = os.path.splitext(path)[1].lower()
    return ext in [".pdf", ".doc", ".docx", ".ppt", ".pptx", ".xls", ".xlsx", ".txt"]

In [37]:
# 이미지 전처리 함수
MAX_SIDE = 8000           # 최대 너비/높이 (px)
MAX_FILE_MB = 3.75        # 최대 파일 크기 (MB)
MAX_FILE_B = int(MAX_FILE_MB * 1024 * 1024)

def compress_image_new(img: Image.Image, target_bytes=MAX_FILE_B) -> bytes:
    """이미지를 JPEG로 압축하여 지정된 크기 이하로 만듦"""
    img = img.convert("RGB")  # JPEG 저장을 위해 RGB 변환
    buf, quality = io.BytesIO(), 95
    
    img.save(buf, format="JPEG", quality=quality, optimize=True)
    while buf.getbuffer().nbytes > target_bytes and quality > 10:
        quality -= 5
        buf = io.BytesIO()
        img.save(buf, format="JPEG", quality=quality, optimize=True)
    
    return buf.getvalue()

def resize_if_needed(img: Image.Image) -> Image.Image:
    """너비나 높이가 8000px 초과 시 리사이즈"""
    if max(img.size) <= MAX_SIDE:
        return img
    
    ratio = MAX_SIDE / max(img.size)
    new_size = (int(img.width * ratio), int(img.height * ratio))
    return img.resize(new_size, Image.Resampling.LANCZOS)

def vertical_crops(img: Image.Image) -> List[Image.Image]:
    """높이 8000px 초과 시 여러 장으로 분할"""
    if img.height <= MAX_SIDE:
        return [img]
    
    crops = []
    for top in range(0, img.height, MAX_SIDE):
        bottom = min(top + MAX_SIDE, img.height)
        crops.append(img.crop((0, top, img.width, bottom)))
    
    return crops

def preprocess_image(path: str, save_crops: bool = True, save_dir: str = "processed_images") -> List[Tuple[bytes, str]]:
    """이미지를 Bedrock API 제한에 맞게 전처리하고 크롭된 이미지들을 저장"""
    results = []
    
    # 저장 디렉토리 생성
    if save_crops and not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"저장 디렉토리 생성: {save_dir}")
    
    with Image.open(path) as img:
        print(f"원본 이미지 크기: {img.size}")
        
        # 1단계: 리사이즈
        img = resize_if_needed(img)
        if img.size != Image.open(path).size:
            print(f"리사이즈 후 크기: {img.size}")
        
        # 2단계: 수직 분할
        crops = vertical_crops(img)
        if len(crops) > 1:
            print(f"{len(crops)}개 이미지로 분할됨")
        
        # 3단계: 압축 및 저장
        for i, crop in enumerate(crops, 1):
            # 압축된 바이트 데이터 생성
            img_bytes = compress_image_new(crop)
            results.append((img_bytes, "jpeg"))
            print(f"이미지 {i} 압축 완료: {len(img_bytes)/(1024*1024):.2f}MB")
            
            # 크롭된 이미지를 파일로 저장
            if save_crops:
                # 원본 파일명에서 확장자 제거
                base_name = os.path.splitext(os.path.basename(path))[0]
                save_path = os.path.join(save_dir, f"{base_name}_crop_{i}.jpg")
                
                crop.save(save_path, format="JPEG", quality=95)
                print(f"크롭된 이미지 저장: {save_path}")
    
    return results



## 3. OCR

In [35]:
def run_ocr_multi(images: List[Tuple[bytes, str]], filename: str):
    """여러 이미지를 한 번에 처리하는 OCR"""
    # 한 메시지에 텍스트 + 여러 이미지 포함
    content_items = [
        {"text": f"{filename}\n\n한국어 리플렛의 모든 텍스트를 추출하세요."}
    ]
    
    for img_bytes, fmt in images:
        content_items.append({
            "image": {"format": fmt, "source": {"bytes": img_bytes}}
        })
    
    messages = [{"role": "user", "content": content_items}]
    
    response = client.converse(
        modelId=nova_pro_model_id,
        system=[{"text": ocr_system_prompt}],
        messages=messages,
        inferenceConfig={
            "maxTokens": 4000,
            "temperature": 0.1,
            "topP": 0.9
        }
    )
    return response


In [25]:
def main(path: str):
    if not os.path.exists(path):
        print("❌ 파일이 존재하지 않습니다.")
        return
    
    print(f"파일 크기: {file_size_mb(path):.2f} MB")
    
    if not is_image(path):
        print("❌ 이미지 파일이 아니므로 OCR 대상에서 제외됩니다.")
        return
    
    print("✓ 이미지 파일로 감지되었습니다.")
    
    # 1) 이미지 전처리
    print("\n=== 이미지 전처리 시작 ===")
    try:
        images = preprocess_image(path)
        print(f"전처리 완료: {len(images)}개 이미지 생성")
    except Exception as e:
        print(f"전처리 오류: {e}")
        return
    
    # 2) OCR 실행
    print("\n=== OCR 실행 ===")
    try:
        response = run_ocr_multi(images, os.path.basename(path))
        
        # 결과 출력
        extracted_text = response["output"]["message"]["content"][0]["text"]
        print("\n=== 추출된 텍스트 ===")
        print(extracted_text)
        
        # 추출된 텍스트를 result.txt 파일로 저장
        output_dir = "ocr-results-with-llm"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "result.txt")
        
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(extracted_text)
        
        print(f"\n✅ 추출된 텍스트가 '{output_path}' 파일에 저장되었습니다.")
        
        # 토큰 사용량
        usage = response["usage"]
        print(f"\n=== 토큰 사용량 ===")
        print(f"Input Tokens: {usage['inputTokens']}")
        print(f"Output Tokens: {usage['outputTokens']}")
        
    except ClientError as e:
        print(f"Bedrock API 오류: {e}")
    except Exception as e:
        print(f"OCR 처리 오류: {e}")


In [38]:
# 실행
SAMPLE_DIR = os.path.join(os.getcwd(), "samples")
TARGET_IMG = os.path.join(SAMPLE_DIR, "sample1.jpg")

main(TARGET_IMG)


파일 크기: 1.92 MB
✓ 이미지 파일로 감지되었습니다.

=== 이미지 전처리 시작 ===
저장 디렉토리 생성: processed_images
원본 이미지 크기: (1000, 17717)
리사이즈 후 크기: (451, 8000)
이미지 1 압축 완료: 0.81MB
크롭된 이미지 저장: processed_images/sample1_crop_1.jpg
전처리 완료: 1개 이미지 생성

=== OCR 실행 ===

=== 추출된 텍스트 ===
## Morning Breeze Hand Cream

"아침 햇살처럼 부드러운 크림"

---

## 브랜드 철학

### 자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

### 브랜드 철학

- 자연의 힘을 담은 핸드크림
- 손에 온기를 님아 넘겨주는 핸드크림

### 브랜드 철학

- 자연의 힘을 담은 핸드크림
- 손에 온기를 님아 넘겨주는 핸드크림

---

## 155 Lab

### Tender garden

---

## 브랜드 철학

### 자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

### Morning Breeze Hand Cream

자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

브랜드 철학

자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

---

## Point.1

### 브랜드 철학

자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

## Point.2

### 브랜드 철학

자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

## Point.3

### 브랜드 철학

자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

## Point.4

### 브랜드 철학

자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

## Point.5

### 브랜드 철학

자연의 힘을 담은 핸드크림

손에 온기를 담아 넘겨주는 핸드크림

---

## Key Ingredient

